# Projekt 1.

## Przygotowanie danych.

Domyślnie wczytuję dane ze zmiennymi które arbitralnie sobie wybrałem - zmienne kategoryczne przekształcone do macierzy zer i jedynek (one-hot encoding). Czyli nie ma zmiennej "linia lotnicza", ale jest zmienna "Is_Ryanair", "Is_LOT" - itp., zmienne binarne. Oczywiście zwykłe numeryczne też są. 

In [18]:
import pandas as pd
import numpy as np

# ostateczne dane do przewidywania typu linii lotniczej - low cost czy tradycyjna
dane = pd.read_pickle("dane_klasyfikacja_onehot2.pkl")

dane.head()

,Departure_time,Arrival_time,Flight_time,Price,Num_Layovers,Cabin_bag,Checked_bag,Days_to_departure,layover_duration,Is_Departure_Warszawa,...,Is_Flight_Thursday,Is_Flight_Tuesday,Is_Flight_Wednesday,Is_Extraction_Monday,Is_Extraction_Saturday,Is_Extraction_Sunday,Is_Extraction_Thursday,Is_Extraction_Tuesday,Is_Extraction_Wednesday,Is_low-cost
1,18.33,22.67,4.33,1087,1,0,0,2,1.583333,True,...,False,False,False,False,True,False,False,False,False,True
3,6.08,11.92,5.83,1776,1,0,0,2,1.083333,True,...,False,False,False,False,True,False,False,False,False,True
4,17.00,22.67,5.67,1878,1,1,0,2,2.833333,True,...,False,False,False,False,True,False,False,False,False,False
5,12.75,17.67,4.92,2176,1,1,0,2,1.583333,True,...,False,False,False,False,True,False,False,False,False,False
6,9.50,13.92,4.42,2272,1,1,0,2,1.333333,True,...,False,False,False,False,True,False,False,False,False,False


In [19]:
dane.shape
dane.columns

Index(['Departure_time', 'Arrival_time', 'Flight_time', 'Price',
       'Num_Layovers', 'Cabin_bag', 'Checked_bag', 'Days_to_departure',
       'layover_duration', 'Is_Departure_Warszawa', 'Departure_time',
       'Arrival_time', 'Flight_time', 'Price', 'Num_Layovers', 'Cabin_bag',
       'Checked_bag', 'Days_to_departure', 'layover_duration',
       'Is_Arrival_Londyn', 'Is_Arrival_Paryż', 'Is_Arrival_Rzym',
       'Is_Flight_Monday', 'Is_Flight_Saturday', 'Is_Flight_Sunday',
       'Is_Flight_Thursday', 'Is_Flight_Tuesday', 'Is_Flight_Wednesday',
       'Is_Extraction_Monday', 'Is_Extraction_Saturday',
       'Is_Extraction_Sunday', 'Is_Extraction_Thursday',
       'Is_Extraction_Tuesday', 'Is_Extraction_Wednesday', 'Is_low-cost'],
      dtype='object')

In [20]:
from sklearn.model_selection import train_test_split

# target variable - cena lotu (zł)
X = dane.drop(columns =["Is_low-cost"])
y = dane["Is_low-cost"]

In [21]:
# podział na zbiór treningowy, walidacyjny i testowy - proporcje 80% : 20% : 20% 
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.75)


# konwersja do macierzy numpy
X_train_np = X_train.to_numpy().astype(np.float64)
y_train_np = y_train.to_numpy().astype(np.float64)

X_val_np = X_val.to_numpy().astype(np.float64)
y_val_np = y_val.to_numpy().astype(np.float64)
X_test_np = X_test.to_numpy().astype(np.float64)
y_test_np = y_test.to_numpy().astype(np.float64)


In [22]:
y_val_np.shape

(21098,)

## Testowanie parametrów

Grid Search - przeszukiwanie kombinacji parametrów. 

In [23]:
import itertools
from glob import glob
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
import numpy as np
from perceptron_nn_classifier import BinaryNeuralNetworkClassifier
import os
num_repetitions = 5
num_classes = len(np.unique(y_test_np))

baseline_params = {
    "num_inputs": [X_test_np.shape[1]], # tego nie ruszać
    "num_layers": [[i] for i in range(1,50)], # od 1 do 20 warstw
    "learning_rate": [ 0.1],
    "num_epochs": [2000],
    "activation_function": ["relu"]
}

keys = list(baseline_params.keys())
combinations = list(itertools.product(*(baseline_params[key] for key in keys)))

# ramka danych z parametrami
params_df = pd.DataFrame(data=combinations, columns=keys)

# pusta lista na wyniki
results = []
for i, row in params_df.iterrows():
    try:
        num_inputs = row["num_inputs"]
        num_layers = row["num_layers"]
        learning_rate = row["learning_rate"]
        activation_function = row["activation_function"]
        num_epochs = row["num_epochs"]
        
        # żeby sprawdzić wszystkie parametry metody fit, idź do perceptron_nn.ipynb
        epochs = []
        train_loss = []
        train_accuracy = []
        val_loss = []
        val_accuracy = []
        test_accuracy = []
        test_precision = []
        test_recall = []
        test_f1 = []
        test_log_loss = []
        
        for k in range(num_repetitions):
            print(f"{k+1} powtórzenie dla kombinacji {i+1}/{len(params_df)}")
            mlp = BinaryNeuralNetworkClassifier(
                num_inputs=num_inputs,
                num_layers=num_layers, 
                learning_rate=learning_rate,
                activation_function=activation_function)
            
            mlp.fit(
                X=X_train_np,
                y=y_train_np,  # już w formacie one-hot
                X_val=X_val_np,
                y_val=y_val_np,  # już w formacie one-hot
                num_epochs=num_epochs,
                verbose=True) 
            
            res = mlp.history
            
            # Przewidywania
            pred_classes = mlp.predict(X_test_np)  # klasy (0, 1, 2, ...)
            pred_proba = mlp.predict_proba(X_test_np)  # prawdopodobieństwa
            
            # Zapisanie wyników
            epochs.append(res["epoch"])
            train_loss.append(res["train_loss"])
            train_accuracy.append(res["train_accuracy"])
            val_loss.append(res["val_loss"])
            val_accuracy.append(res["val_accuracy"])
            
            # Metryki testowe
            test_accuracy.append(accuracy_score(y_test_np, pred_classes))
            test_precision.append(precision_score(y_test_np, pred_classes, average='weighted', zero_division=0))
            test_recall.append(recall_score(y_test_np, pred_classes, average='weighted', zero_division=0))
            test_f1.append(f1_score(y_test_np, pred_classes, average='weighted', zero_division=0))
            
            # Log loss (cross-entropy)
            try:
                test_log_loss.append(log_loss(y_test_np, pred_proba))
            except:
                test_log_loss.append(np.nan)  # w przypadku problemów numerycznych
        
        results.append({
            "num_inputs": num_inputs,
            "num_classes": num_classes,
            "num_layers": num_layers,
            "learning_rate": learning_rate,
            "num_epochs": epochs,  # średnia liczba epok (może się różnić przez early stopping)
            "activation_function": activation_function,
            "train_loss": np.mean(train_loss),
            "train_accuracy": np.mean(train_accuracy),
            "val_loss": np.mean(val_loss),
            "val_accuracy": np.mean(val_accuracy),
            "test_accuracy": np.mean(test_accuracy),
            "test_precision": np.mean(test_precision),
            "test_recall": np.mean(test_recall),
            "test_f1": np.mean(test_f1),
            "test_log_loss": np.mean(test_log_loss)
        })
        
        print(f"Ukończono kombinację {i+1}/{len(params_df)}: "
              f"Test Accuracy = {np.mean(test_accuracy):.4f} ± {np.std(test_accuracy):.4f}")
              
    except KeyboardInterrupt:
        print("Przerwano przez użytkownika")
        break

result_df = pd.DataFrame(results)

# Ensure the folder exists and save the file inside 'klasyfikacja'

os.makedirs("klasyfikacja", exist_ok=True)
files = glob("klasyfikacja/nn_klasyfikacja_*.xlsx")
filename = f"klasyfikacja/nn_klasyfikacja_{len(files)+1}.xlsx"
result_df.to_excel(filename, index=False)
print(f"Wyniki zapisane do pliku: {filename}")

1 powtórzenie dla kombinacji 1/49
Architektura sieci binarnej: Wejścia(34) -> Warstwa_1(1) -> Wyjście(1 neuron - klasyfikacja binarna)
Early stopping na epoce 59/2000. 50 epok bez poprawy. Najlepszy val_loss: 0.4970
2 powtórzenie dla kombinacji 1/49
Architektura sieci binarnej: Wejścia(34) -> Warstwa_1(1) -> Wyjście(1 neuron - klasyfikacja binarna)
Early stopping na epoce 62/2000. 50 epok bez poprawy. Najlepszy val_loss: 0.4970
3 powtórzenie dla kombinacji 1/49
Architektura sieci binarnej: Wejścia(34) -> Warstwa_1(1) -> Wyjście(1 neuron - klasyfikacja binarna)
Early stopping na epoce 68/2000. 50 epok bez poprawy. Najlepszy val_loss: 0.4970
4 powtórzenie dla kombinacji 1/49
Architektura sieci binarnej: Wejścia(34) -> Warstwa_1(1) -> Wyjście(1 neuron - klasyfikacja binarna)
Early stopping na epoce 63/2000. 50 epok bez poprawy. Najlepszy val_loss: 0.4970
5 powtórzenie dla kombinacji 1/49
Architektura sieci binarnej: Wejścia(34) -> Warstwa_1(1) -> Wyjście(1 neuron - klasyfikacja binarna)
E